In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng

from simulator.objects.market import Market
from simulator.objects.policies.ml_policy import MLPolicy, PassiveAggressivePolicy
from simulator.objects.policies.nn_policy import NNPolicy, PriceNaiveNNPolicy
from simulator.utils.utils import (
    generate_participants,
    generate_stocks,
)

rng = default_rng()

# With price-naive Perceptron

In [2]:
N_STOCKS = 100
N_TRIALS = 25
N_TIMESTEPS = 365
FEATURES = [
    "price",
    "cash",
    "earning value of assets",
    "latest quarterly earnings",
    "1-day percent change",
    "5-day percent change",
    "10-day percent change",
    "1-month percent change",
    "3-month percent change",
    "6-month percent change",
    "1-year percent change",
    "3-year percent change",
    "5-year percent change",
]

nn1_scores = []
nn2_scores = []
random_forest_scores = []
passive_aggressive_scores = []

feature_importance_history = []

market_value_history = []

for trial_ind in range(N_TRIALS):
    market = Market(stocks=generate_stocks(N_STOCKS), interest_rate_apy=0.02)

    market.add_participants(
        generate_participants(
            20,
            market.stocks,
            PriceNaiveNNPolicy,
            [market, 30, 10, Path("model_naive_price.pt"), 0.05],
        )
    )

    market.add_participants(
        generate_participants(
            20,
            market.stocks,
            NNPolicy,
            [market, 30, 10, Path("model_high_prices.pt"), 0.05],
        )
    )

    market.add_participants(
        generate_participants(
            20,
            market.stocks,
            MLPolicy,
            [
                market,
                50,
                10,
                Path("random_forest_regressor.pkl"),
            ],
        )
    )

    market.add_participants(
        generate_participants(
            20,
            market.stocks,
            PassiveAggressivePolicy,
            [market, 50, 10, Path("passive_aggressive.pkl")],
        )
    )

    print("TRIAL INDEX:", trial_ind)

    for i in range(N_TIMESTEPS):
        print(i)
        market.step_market()

    percent_changes = []
    for i in range(len(market.participants)):
        percent_changes.append(
            (
                market.participants[i].total_value[-1]
                - market.participants[i].total_value[-N_TIMESTEPS]
            )
            / market.participants[i].total_value[-N_TIMESTEPS]
        )

    percent_changes_array = np.array(percent_changes)
    relative_performance = percent_changes_array - np.mean(percent_changes_array)

    nn_1_score = np.mean(relative_performance[:20])
    nn_2_score = np.mean(relative_performance[20:40])
    random_forest_score = np.mean(relative_performance[40:60])
    passive_aggressive_score = np.mean(relative_performance[60:])

    print(f"NN1: {nn_1_score}")
    print(f"NN2: {nn_2_score}")
    print(f"random forest: {random_forest_score}")
    print(f"passive aggressive: {passive_aggressive_score}")

    nn1_scores.append(nn_1_score)
    nn2_scores.append(nn_2_score)
    random_forest_scores.append(random_forest_score)
    passive_aggressive_scores.append(passive_aggressive_score)

    all_feature_importances = np.zeros_like(
        market.participants[40].policy.valuation_model.feature_importances_
    )
    for i in range(40, 60):
        all_feature_importances += market.participants[
            i
        ].policy.valuation_model.feature_importances_
    avg_feature_importances = all_feature_importances / 20

    feature_importance_map = {
        FEATURES[i]: float(avg_feature_importances[i]) for i in range(len(FEATURES))
    }
    print(feature_importance_map)

    feature_importance_history.append(avg_feature_importances)

    market_value_history.append(market.market_value_history)
    print(market.market_value_history)


TRIAL INDEX: 0
0
Number of buy order stocks: 59
Number of sell order stocks: 69
Market total value: 9688.219026518087
1
Number of buy order stocks: 56
Number of sell order stocks: 73
Market total value: 12641.224718775778
2
Number of buy order stocks: 54
Number of sell order stocks: 81
Market total value: 14986.343482684628
3
Number of buy order stocks: 61
Number of sell order stocks: 77
Market total value: 16458.09200306835
4
Number of buy order stocks: 59
Number of sell order stocks: 78
Market total value: 17862.661936041837
5
Number of buy order stocks: 60
Number of sell order stocks: 85
Market total value: 19000.14153266951
6
Number of buy order stocks: 57
Number of sell order stocks: 81
Market total value: 20004.585338136763
7
Number of buy order stocks: 57
Number of sell order stocks: 87
Market total value: 20366.264063258997
8
Number of buy order stocks: 58
Number of sell order stocks: 85
Market total value: 20807.479758955968
9
Number of buy order stocks: 58
Number of sell orde

In [6]:
all_feature_importances = np.zeros_like(
    market.participants[40].policy.valuation_model.feature_importances_
)
for i in range(40, 60):
    all_feature_importances += market.participants[
        i
    ].policy.valuation_model.feature_importances_
avg_feature_importances = all_feature_importances / 20
FEATURES = [
    "price",
    "cash",
    "earning value of assets",
    "latest quarterly earnings",
    "1-day percent change",
    "5-day percent change",
    "10-day percent change",
    "1-month percent change",
    "3-month percent change",
    "6-month percent change",
    "1-year percent change",
    "3-year percent change",
    "5-year percent change",
]

feature_importance_map = {
    FEATURES[i]: float(avg_feature_importances[i]) for i in range(len(FEATURES))
}

print(feature_importance_map)

{'price': 0.31879417413793576, 'cash': 0.2128053880376565, 'earning value of assets': 0.03407903743925819, 'latest quarterly earnings': 0.011752022999835767, '1-day percent change': 0.006675070421479368, '5-day percent change': 0.02450486280000874, '10-day percent change': 0.011826214251209364, '1-month percent change': 0.01984740920395044, '3-month percent change': 0.09490122984395656, '6-month percent change': 0.10884492768666516, '1-year percent change': 0.05836577477231828, '3-year percent change': 0.05754802340744607, '5-year percent change': 0.03593578017567799}


# No price naive Perceptron

In [7]:
N_STOCKS = 100
N_TRIALS = 25
N_TIMESTEPS = 365
FEATURES = [
    "price",
    "cash",
    "earning value of assets",
    "latest quarterly earnings",
    "1-day percent change",
    "5-day percent change",
    "10-day percent change",
    "1-month percent change",
    "3-month percent change",
    "6-month percent change",
    "1-year percent change",
    "3-year percent change",
    "5-year percent change",
]

nn1_scores2 = []
nn2_scores2 = []
random_forest_scores2 = []
passive_aggressive_scores2 = []

feature_importance_history2 = []

market_value_history2 = []

for trial_ind in range(N_TRIALS):
    market = Market(stocks=generate_stocks(N_STOCKS), interest_rate_apy=0.02)

    market.add_participants(
        generate_participants(
            40,
            market.stocks,
            NNPolicy,
            [market, 30, 10, Path("model_high_prices.pt"), 0.05],
        )
    )

    market.add_participants(
        generate_participants(
            20,
            market.stocks,
            MLPolicy,
            [
                market,
                50,
                10,
                Path("random_forest_regressor.pkl"),
            ],
        )
    )

    market.add_participants(
        generate_participants(
            20,
            market.stocks,
            PassiveAggressivePolicy,
            [market, 50, 10, Path("passive_aggressive.pkl")],
        )
    )

    print("TRIAL INDEX:", trial_ind)

    for i in range(N_TIMESTEPS):
        print(i)
        market.step_market()

    percent_changes = []
    for i in range(len(market.participants)):
        percent_changes.append(
            (
                market.participants[i].total_value[-1]
                - market.participants[i].total_value[-N_TIMESTEPS]
            )
            / market.participants[i].total_value[-N_TIMESTEPS]
        )

    percent_changes_array = np.array(percent_changes)
    relative_performance = percent_changes_array - np.mean(percent_changes_array)

    nn_1_score = np.mean(relative_performance[:20])
    nn_2_score = np.mean(relative_performance[20:40])
    random_forest_score = np.mean(relative_performance[40:60])
    passive_aggressive_score = np.mean(relative_performance[60:])

    print(f"NN1: {nn_1_score}")
    print(f"NN2: {nn_2_score}")
    print(f"random forest: {random_forest_score}")
    print(f"passive aggressive: {passive_aggressive_score}")

    nn1_scores2.append(nn_1_score)
    nn2_scores2.append(nn_2_score)
    random_forest_scores2.append(random_forest_score)
    passive_aggressive_scores2.append(passive_aggressive_score)

    all_feature_importances = np.zeros_like(
        market.participants[40].policy.valuation_model.feature_importances_
    )
    for i in range(40, 60):
        all_feature_importances += market.participants[
            i
        ].policy.valuation_model.feature_importances_
    avg_feature_importances = all_feature_importances / 20

    feature_importance_map = {
        FEATURES[i]: float(avg_feature_importances[i]) for i in range(len(FEATURES))
    }
    print(feature_importance_map)

    feature_importance_history2.append(avg_feature_importances)

    market_value_history2.append(market.market_value_history)
    print(market.market_value_history)

TRIAL INDEX: 0
0
Number of buy order stocks: 50
Number of sell order stocks: 66
Market total value: 9512.32986909008
1
Number of buy order stocks: 54
Number of sell order stocks: 70
Market total value: 11049.430333279257
2
Number of buy order stocks: 53
Number of sell order stocks: 77
Market total value: 12734.816535272415
3
Number of buy order stocks: 49
Number of sell order stocks: 74
Market total value: 15418.01514309278
4
Number of buy order stocks: 58
Number of sell order stocks: 78
Market total value: 16461.437089731277
5
Number of buy order stocks: 52
Number of sell order stocks: 81
Market total value: 17705.664488852814
6
Number of buy order stocks: 54
Number of sell order stocks: 77
Market total value: 18560.37714290582
7
Number of buy order stocks: 55
Number of sell order stocks: 79
Market total value: 19134.381415759875
8
Number of buy order stocks: 53
Number of sell order stocks: 80
Market total value: 19715.578302291662
9
Number of buy order stocks: 51
Number of sell order